In [2]:
cd ../../../datasets/190829_Kanshan_zjfx

/home/zengrui/datasets/190829_Kanshan_zjfx


In [3]:
ls

train.csv  val.csv


## 0.1 超参数

In [49]:
import numpy as np
import pandas as pd
# import pymysql
import torch
from tqdm import tqdm

BATCH_SIZE = 65536
DB_NAME = 'zhihu2019_dataset'
DO_DATASET_BALANCE = True
EPOCH = 5
LR = 0.0001
TRAIN_ALL = True

PKL_DIR_READ = '../../jupyter/190919_DataMiningHW/project3/storage/train_param.pkl'
PKL_DIR_OUT = '../../jupyter/190919_DataMiningHW/project3/storage/train_param.pkl'
RESULT_DIR_OUT = '../../jupyter/190919_DataMiningHW/project3/storage/result.txt'

DEVICE = torch.device('cuda:3') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda', index=3)

## 0.2 数据库函数

In [3]:
class DB():
    def __init__(self, host='localhost', port=3306, db=DB_NAME, user='rayi', passwd='123456', charset='utf8'):
        self.conn = pymysql.connect(host=host, port=port, db=db, user=user, passwd=passwd, charset=charset)   
        self.cur = self.conn.cursor(cursor = pymysql.cursors.DictCursor)

    def __enter__(self): 
        return self.cur

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.conn.commit()
        self.cur.close()
        self.conn.close()

## 1.1 加载数据集

取训练集。

In [7]:
# with DB() as db:
#     db.execute('select * from train')
#     train = pd.DataFrame(db.fetchall())

train = pd.read_csv('train.csv', header=0, sep=',')

取测试集。

In [8]:
# with DB() as db:
#     db.execute('select * from val')
#     test = pd.DataFrame(db.fetchall())

test = pd.read_csv('val.csv', header=0, sep=',')

合并二集。

In [9]:
col_list = ['q_id', 'u_id', 'do_answer', 'invite_hour', 'follow_topic_hit', 
             'interest_topic_hit', 'sex', 'access_freq', 'bin_feat_a', 'bin_feat_b', 
             'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'multi_feat_a', 'multi_feat_b',
             'multi_feat_c', 'multi_feat_d', 'multi_feat_e', 'salt', 'u_answer', 
             'u_good_answer', 'u_recommand_answer', 'u_image_answer', 'u_video_answer', 
             'u_word_avg', 'u_like_avg', 'u_unlike_avg', 'u_comment_avg', 
             'u_collect_avg', 'u_thanks_avg', 'u_report_avg','u_nohelp_avg', 
             'u_oppose_avg', 'q_answer', 'q_good_answer', 'q_recommand_answer',
             'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 
             'q_collect', 'q_thanks']
# train = train[col_list]
# test = test[col_list]

In [10]:
data = pd.concat([train, test], axis=0)
data.columns

Index(['q_id', 'u_id', 'do_answer', 'invite_hour', 'follow_topic_hit',
       'interest_topic_hit', 'sex', 'access_freq', 'bin_feat_a', 'bin_feat_b',
       'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'multi_feat_a',
       'multi_feat_b', 'multi_feat_c', 'multi_feat_d', 'multi_feat_e', 'salt',
       'u_answer', 'u_good_answer', 'u_recommand_answer', 'u_image_answer',
       'u_video_answer', 'u_word_avg', 'u_like_avg', 'u_unlike_avg',
       'u_comment_avg', 'u_collect_avg', 'u_thanks_avg', 'u_report_avg',
       'u_nohelp_avg', 'u_oppose_avg', 'q_answer', 'q_good_answer',
       'q_recommand_answer', 'q_image_answer', 'q_video_answer', 'q_word_avg',
       'q_like', 'q_comment', 'q_collect', 'q_thanks'],
      dtype='object')

## 1.2 处理数据集

离散特征的`LabelEncoder`编码。

In [11]:
from sklearn.preprocessing import LabelEncoder

# 离散值编码
class_feat = [#'用户id','问题id',
              'sex', 'access_freq','multi_feat_a','multi_feat_b',
              'multi_feat_c','multi_feat_d','multi_feat_e']
encoder = LabelEncoder()
for i in class_feat:
    data[i] = encoder.fit_transform(data[i])
    
data.iloc[:10]

,q_id,u_id,do_answer,invite_hour,follow_topic_hit,interest_topic_hit,sex,access_freq,bin_feat_a,bin_feat_b,...,q_answer,q_good_answer,q_recommand_answer,q_image_answer,q_video_answer,q_word_avg,q_like,q_comment,q_collect,q_thanks
0,Q2166419046,M401693808,0,1.558090,1.308280,-0.282984,2,4,-1.218040,1.485950,...,-0.162667,-0.03874,-0.007758,-0.099289,-0.038624,-0.268488,-0.030466,-0.052246,-0.015932,-0.023172
1,Q1550017551,M3392373099,0,-0.387527,-0.438202,-0.282984,2,1,0.820988,-0.672968,...,0.298958,-0.03874,-0.007758,-0.099289,-0.038624,-0.175978,-0.030466,-0.040992,-0.015932,-0.023172
2,Q604029601,M2317670257,0,0.319970,-0.438202,-0.282984,2,4,0.820988,-0.672968,...,-0.162667,-0.03874,-0.007758,-0.099289,-0.038624,-0.268488,-0.030466,-0.052246,-0.015932,-0.023172
3,Q2350061229,M1618461867,0,-0.387527,-0.438202,2.440210,2,0,0.820988,-0.672968,...,-0.096721,-0.03874,-0.007758,-0.099289,-0.038624,0.705070,-0.028959,-0.018486,-0.013906,-0.014543
4,Q2443223942,M3544409350,0,-1.625650,-0.438202,-0.282984,2,1,0.820988,-0.672968,...,3.926020,-0.03874,-0.007758,1.899240,-0.038624,-0.175978,-0.012378,-0.029739,-0.015932,-0.023172
5,Q640765464,M2818659842,0,0.496844,1.308280,-0.282984,1,0,0.820988,-0.672968,...,-0.162667,-0.03874,-0.007758,-0.099289,-0.038624,-0.268488,-0.030466,-0.052246,-0.015932,-0.023172
6,Q795459266,M2818659842,0,1.204340,-0.438202,-0.282984,1,0,0.820988,-0.672968,...,0.101119,-0.03874,-0.007758,0.300416,-0.038624,0.224899,-0.022929,0.004021,-0.015932,-0.023172
7,Q190554387,M1581217469,1,-0.918149,-0.438202,-0.282984,2,1,0.820988,-0.672968,...,-0.030774,-0.03874,-0.007758,-0.099289,-0.038624,-0.204612,-0.030466,-0.029739,-0.015932,-0.014543
8,Q1958712851,M3021021791,0,1.027470,-0.438202,-0.282984,2,4,-1.218040,1.485950,...,-0.096721,-0.03874,-0.007758,-0.099289,-0.038624,-0.217828,-0.030466,-0.052246,-0.015932,-0.023172
9,Q311993584,M1766315480,0,0.319970,-0.438202,-0.282984,1,0,0.820988,-0.672968,...,0.101119,-0.03874,-0.007758,-0.099289,-0.038624,0.165428,-0.028959,0.026528,-0.013906,0.002716


In [12]:
data.iloc[0]

q_id                  Q2166419046
u_id                   M401693808
do_answer                       0
invite_hour               1.55809
follow_topic_hit          1.30828
interest_topic_hit      -0.282984
sex                             2
access_freq                     4
bin_feat_a               -1.21804
bin_feat_b                1.48595
bin_feat_c              -0.174441
bin_feat_d              -0.416016
bin_feat_e              -0.239396
multi_feat_a                 1917
multi_feat_b                  170
multi_feat_c                  244
multi_feat_d                  799
multi_feat_e                    1
salt                    -0.600888
u_answer                -0.284904
u_good_answer          -0.0244919
u_recommand_answer    -0.00454287
u_image_answer          -0.246062
u_video_answer         -0.0619912
u_word_avg              -0.258716
u_like_avg             -0.0296191
u_unlike_avg            -0.034083
u_comment_avg          -0.0550066
u_collect_avg            -0.01177
u_thanks_avg  

## 1.3 包装数据集

In [16]:
# import numpy as np

# if DO_DATASET_BALANCE:
#     temp = data[data['do_answer'] == 1]  # 用于均衡的数据
#     print('用于均衡的数据有 %d 条.' % len(temp))
#     train_x = data[data['do_answer']>-1].append([temp, temp, temp])
# else:
#     train_x = data[data['do_answer']>-1]
# train_x, train_y = train_x[col_list[3:]].values, train_x[['do_answer']].values[:, 0]

test_x = data[data['do_answer']==-1]
test_x, test_y = test_x[col_list[3:]].values, test_x[['do_answer']].values[:, 0]
# print('ok.')

In [13]:
# 正负样本df
positive_samples = data[data['do_answer'] == 1]
negative_samples = data[data['do_answer'] == 0]
print('pos_len:', len(positive_samples))
print('neg_len:', len(negative_samples))

pos_len: 1682914
neg_len: 7806219


In [17]:
from torch.utils.data import Dataset, DataLoader


class MyDataset(Dataset):
    """每次new都会随机均衡数据集。"""
    def __init__(self, mode='train'):
        self.mode = mode
        if mode == 'train':
            train_x = pd.concat([
                positive_samples,
                negative_samples.sample(n=len(positive_samples), axis=0)
            ], axis=0)
            train_x, train_y = train_x[col_list[3:]].values, train_x[['do_answer']].values[:, 0]
            self.data = {
                'x': train_x,
                'y': train_y,
            }
        elif mode == 'test':
            self.data = {
                'x': test_x,
                'y': test_y,
            }
        else:
            raise
            
    def __getitem__(self, index):
        return self.data['x'][index], self.data['y'][index]
        
    def __len__(self):
        return len(self.data['x'])
    
    
print('ok.')

ok.


In [41]:
import math
from torch.utils.data import random_split

train_size, val_size, test_size = 0, 0, 0
train_len, val_len, test_len = 0, 0, 0
data_loader = {}

def flush_dataloader(log=True):
    global train_size, val_size, test_size, train_len, val_len, test_len, data_loader
    
    full_train_dataset = MyDataset(mode='train')
    test_dataset = MyDataset(mode='test')

    if TRAIN_ALL:
        data_loader = {
            'train': DataLoader(dataset=full_train_dataset, 
                                batch_size=BATCH_SIZE, 
                                num_workers=5,
                                shuffle=True),
            'test': DataLoader(dataset=test_dataset,
                               batch_size=BATCH_SIZE,
                               num_workers=5,
                               shuffle=False),
        }
        train_size = len(full_train_dataset)
        test_size = len(test_dataset)
    else:
        train_size = int(0.8 * len(full_train_dataset))
        val_size = len(full_train_dataset) - train_size
        train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

        data_loader = {
            'train': DataLoader(dataset=train_dataset, 
                                batch_size=BATCH_SIZE, 
                               num_workers=5,
                                shuffle=True),
            'val': DataLoader(dataset=val_dataset, 
                              batch_size=BATCH_SIZE,
                               num_workers=5,
                              shuffle=False),
            'test': DataLoader(dataset=test_dataset,
                               batch_size=BATCH_SIZE,
                               num_workers=5,
                               shuffle=False),
        }
        train_size = len(train_dataset)
        val_size = len(val_dataset)
        test_size = len(test_dataset)

    if log:
        print('train_size:', train_size)
        print('val_size:', val_size)
        print('test_size:', test_size)

    # 计算一轮enumerate的长度
    train_len = math.ceil(train_size / BATCH_SIZE)
    val_len = math.ceil(val_size / BATCH_SIZE)
    test_len = math.ceil(test_size / BATCH_SIZE)

    if log:
        print('train_len:', train_len)
        print('val_len:', val_len)
        print('test_len:', test_len)

## 2.1 构建网络

In [30]:
from my_model import MyNet_v2

net = MyNet_v2(50).to(DEVICE)
net

MyNet_v2(
  (embed_sex): Embedding(3, 2)
  (embed_access_freq): Embedding(5, 2)
  (embed_multi_a): Embedding(2561, 13)
  (embed_multi_b): Embedding(291, 6)
  (embed_multi_c): Embedding(428, 7)
  (embed_multi_d): Embedding(1556, 11)
  (embed_multi_e): Embedding(2, 1)
  (bn0): BatchNorm1d(85, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn6): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn7): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn8): Batch

## 2.2 优化器、损失函数

In [50]:
from torch import optim, nn

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.75)

'ok'

'ok'

## 2.3 训练

In [54]:
from tqdm import tqdm

for epoch in range(EPOCH):
    count, running_correct, running_loss = 0, 0, 0
    flush_dataloader(log=False)
    
    net.train()
    with tqdm(total=train_len) as pbar:
        for step, (bx, by) in enumerate(data_loader['train']):
            # 训练
            bx = bx.float().to(DEVICE)
            by = by.long().to(DEVICE)

            prediction = net(bx)
            loss = loss_function(prediction, by)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 统计成效
            pre = torch.argmax(torch.softmax(prediction, dim=1), dim=1)
            my_loss = float(loss.data.cpu().numpy())
            my_correct = sum(pre.cpu().numpy() == by.cpu().numpy())
            my_acc = my_correct / BATCH_SIZE
            count += 1
            running_loss += my_loss
            running_correct += my_correct
            
            # 更新进度条
            pbar.update(1)
            pbar.set_description('Epoch: %d, Loss: %.4f, Acc: %.4f' % (
                                  epoch, 
                                  my_loss, 
                                  my_acc))
    
    # 输出一轮结果
    print('Epoch: %d, Loss: %.4f, Acc: %.4f, Lr: %.5f' % (
           epoch,
           running_loss / count,
           running_correct / train_size,
           optimizer.state_dict()['param_groups'][0]['lr']))
    
    # 保存参数
    torch.save(net.state_dict(), PKL_DIR_OUT)
    
    # lr scheduler
#     scheduler.step()

Epoch: 0, Loss: 0.4804, Acc: 0.2720: 100%|██████████| 52/52 [00:26<00:00,  1.95it/s]


Epoch: 0, Loss: 0.4772, Acc: 0.7616, Lr: 0.00010


Epoch: 1, Loss: 0.4758, Acc: 0.2726: 100%|██████████| 52/52 [00:26<00:00,  1.95it/s]


Epoch: 1, Loss: 0.4769, Acc: 0.7618, Lr: 0.00010


Epoch: 2, Loss: 0.4843, Acc: 0.2705: 100%|██████████| 52/52 [00:26<00:00,  1.94it/s]


Epoch: 2, Loss: 0.4773, Acc: 0.7615, Lr: 0.00010


Epoch: 3, Loss: 0.4783, Acc: 0.2737: 100%|██████████| 52/52 [00:27<00:00,  1.92it/s]


Epoch: 3, Loss: 0.4768, Acc: 0.7619, Lr: 0.00010


Epoch: 4, Loss: 0.4788, Acc: 0.2729: 100%|██████████| 52/52 [00:26<00:00,  1.94it/s]

Epoch: 4, Loss: 0.4768, Acc: 0.7618, Lr: 0.00010


训练结果：

```
Epoch: 0, Loss: 0.5568, Acc: 0.7027, Lr: 0.10000
Epoch: 1, Loss: 0.5199, Acc: 0.7327, Lr: 0.10000
Epoch: 2, Loss: 0.5118, Acc: 0.7379, Lr: 0.10000
Epoch: 3, Loss: 0.5062, Acc: 0.7421, Lr: 0.10000
Epoch: 4, Loss: 0.5023, Acc: 0.7449, Lr: 0.10000
Epoch: 5, Loss: 0.4985, Acc: 0.7477, Lr: 0.10000
Epoch: 6, Loss: 0.4967, Acc: 0.7493, Lr: 0.10000
Epoch: 7, Loss: 0.4943, Acc: 0.7508, Lr: 0.10000
Epoch: 8, Loss: 0.4929, Acc: 0.7520, Lr: 0.10000
Epoch: 9, Loss: 0.4914, Acc: 0.7529, Lr: 0.10000
Epoch: 0, Loss: 0.4852, Acc: 0.7569, Lr: 0.10000
Epoch: 1, Loss: 0.4838, Acc: 0.7575, Lr: 0.10000
Epoch: 2, Loss: 0.4830, Acc: 0.7582, Lr: 0.10000
Epoch: 0, Loss: 0.4806, Acc: 0.7596, Lr: 0.10000
Epoch: 1, Loss: 0.4808, Acc: 0.7595, Lr: 0.10000
Epoch: 2, Loss: 0.4802, Acc: 0.7596, Lr: 0.10000
Epoch: 0, Loss: 0.4778, Acc: 0.7614, Lr: 0.01000
Epoch: 1, Loss: 0.4775, Acc: 0.7613, Lr: 0.01000
Epoch: 2, Loss: 0.4773, Acc: 0.7615, Lr: 0.01000
Epoch: 0, Loss: 0.4769, Acc: 0.7618, Lr: 0.00100
Epoch: 1, Loss: 0.4770, Acc: 0.7616, Lr: 0.00100
Epoch: 2, Loss: 0.4771, Acc: 0.7615, Lr: 0.00100
Epoch: 0, Loss: 0.4768, Acc: 0.7619, Lr: 0.00010
Epoch: 1, Loss: 0.4775, Acc: 0.7614, Lr: 0.00010
Epoch: 2, Loss: 0.4773, Acc: 0.7615, Lr: 0.00010
Epoch: 3, Loss: 0.4770, Acc: 0.7616, Lr: 0.00010
Epoch: 4, Loss: 0.4770, Acc: 0.7618, Lr: 0.00010
Epoch: 0, Loss: 0.4769, Acc: 0.7617, Lr: 0.00010
Epoch: 1, Loss: 0.4770, Acc: 0.7617, Lr: 0.00010
Epoch: 2, Loss: 0.4773, Acc: 0.7616, Lr: 0.00010
Epoch: 3, Loss: 0.4772, Acc: 0.7615, Lr: 0.00010
Epoch: 4, Loss: 0.4770, Acc: 0.7618, Lr: 0.00010
Epoch: 0, Loss: 0.4772, Acc: 0.7616, Lr: 0.00010
Epoch: 1, Loss: 0.4769, Acc: 0.7618, Lr: 0.00010
Epoch: 2, Loss: 0.4773, Acc: 0.7615, Lr: 0.00010
Epoch: 3, Loss: 0.4768, Acc: 0.7619, Lr: 0.00010
Epoch: 4, Loss: 0.4768, Acc: 0.7618, Lr: 0.00010
```

## 2.4 存储网络

In [55]:
torch.save(net.state_dict(), PKL_DIR_OUT)

## 4.1 导出 test 结果

In [57]:
result = data[data['do_answer'] == -1]
result.iloc[-1]

q_id                  Q2590286630
u_id                  M3168697168
do_answer                      -1
invite_hour             -0.741275
follow_topic_hit        -0.438202
interest_topic_hit      -0.282984
sex                             1
access_freq                     0
bin_feat_a               0.820988
bin_feat_b              -0.672968
bin_feat_c              -0.174441
bin_feat_d              -0.416016
bin_feat_e              -0.239396
multi_feat_a                 1092
multi_feat_b                  115
multi_feat_c                    6
multi_feat_d                  980
multi_feat_e                    1
salt                      3.34578
u_answer                  0.32472
u_good_answer          -0.0244919
u_recommand_answer    -0.00454287
u_image_answer            2.30773
u_video_answer         -0.0619912
u_word_avg                 2.3691
u_like_avg               0.239419
u_unlike_avg              1.31014
u_comment_avg             2.19619
u_collect_avg            0.177103
u_thanks_avg  

In [60]:
from tqdm import tqdm
from my_model import MyNet_v2

net = MyNet_v2(50).to(DEVICE)
if torch.cuda.is_available():
    net.load_state_dict(torch.load(PKL_DIR_READ))
else:
    net.load_state_dict(torch.load(PKL_DIR_READ, map_location='cpu'))
net.eval()

res_list = []
with tqdm(total=test_len) as pbar:
    for step, (bx, by) in enumerate(data_loader['test']):
        bx = bx.float().to(DEVICE)
        by = by.long().to(DEVICE)
        
        prediction = net(bx)
        
        # 计算结果
        pre = torch.argmax(torch.softmax(prediction, dim=1), dim=1)
        res_list.extend(pre.cpu().numpy())
        pbar.update(1)
        
result.loc[:, 'do_answer'] = res_list
result[['do_answer']] = result[['do_answer']].astype(int)  # 数据类型转换为int

'ok'

100%|██████████| 18/18 [00:04<00:00,  4.12it/s]
/home/zengrui/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/zengrui/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


'ok'

In [64]:
invite_info_evaluate = pd.read_csv('invite_info_evaluate_1_0926.txt', 
                                   header=None, sep='\t')
invite_info_evaluate.columns = ['q_id1', 'u_id1', 'invite_time']

In [66]:
result_out = pd.concat([result[['q_id', 'u_id', 'do_answer']], invite_info_evaluate],
                       axis=1)
len(result_out)

1141683

In [67]:
result_out[['q_id', 'u_id', 'invite_time', 
        'do_answer']].to_csv(RESULT_DIR_OUT, index=False, header=False, sep='\t')  # 导出数据

'ok'

'ok'